# Домашнее задание
Анализ данных

# Pro

Возьмите базу квартир (moscow.csv). Проведите парсинг и очистку данных. Проанализируйте зависимость стоимости квартир от различных параметров (количество комнат, станции метро и т. д.). Постройте графики.


--- ЗАГРУЗКА МОДУЛЕЙ И БИБЛИОТЕК

In [1]:
import gdown                                                         # Загрузка файлов из облака
import pandas as pd                                                  # Загрузка библотеки pandas
from pylab import rcParams
import seaborn as sns                                                # seaborn для графиков
import numpy as np                                                   # Загрузка библиотеки для работы с массивами данных
import matplotlib.pyplot as plt                                      # Загрузка библиотеки для построения графиковн7г
import re                                                            # Регулярные выражения
# Команда для отображения графики в ячейках
%matplotlib inline

--- ЗАГРУЗКА, ОБРАБОТКА БД КОПИЯ

In [ ]:
                                                                     # Обработка базы
                                                                     # Загружаем базу данных с гугл драйв
gdown.download('https://drive.google.com/uc?id=1dmrOlzMfy_7RkveIHRxBD6wmdgqi_Gfh', None, quiet=True)
                                                                     # Создадаю переменную data, в которую с помощью функции pd.read_csv() загружаю данные из файла
data = pd.read_csv('moscow.csv', sep=";")   
data_copy = data.copy()                                              # Создайте копию датасета, чтобы избежать изменения оригинального
data_copy.head(5)                                                    # Ввыводим первые 10 строк базы данных

--- РЕДАКТИРОВАНИЕ БД ЗАМЕНА, УДАЛЕНИЕ ЭЛЕМЕНТОВ

In [ ]:
df = data_copy[data_copy['Комнат'] != 'Для заметок:']               # Сохраняем в переменную df новый датафрейм без строк для заметок
df = df.reset_index(drop = True)                                    # Переиндексация строк в базе данных
del df['ГРМ']                                                       # Удаляем столбец ГРМ
del df['Примечание']
df['Балкон'].fillna('Отсутствует', inplace=True)                    # Заменяем NaN в столбце Балкон на Отсутствует
df['Санузел'].fillna('Отсутствует', inplace=True)                   # Заменяем NaN На отсутствует в колонкеСанузел
df.loc[df['Бонус агенту'].isna(), 'Бонус агенту'] = 0               # Задаем значение 0 для Nan-значений в столбце Бонус агенту
df.head(5)

--- РЕДАКТИРОВАНИЕ БД ЗАМЕНА, УДАЛЕНИЕ ЭЛЕМЕНТОВ

In [5]:
# Меняем значения NaN на 0 а остальные значения приводим в нормальный вид
pd.options.mode.chained_assignment = None                           # Отключаем warnings
bonus = df['Бонус агенту']                                          # Помещаем столбец бонус агенту в переменную - bonus
coast = df['Цена, руб.']                                            # Помещаем столбец ценаб руб. в переменную - coast
            
for i in range(len(bonus)):                                         # Проходим по всем значениям столбца Бонус агенту
    if isinstance(bonus[i], str) and 'руб.' in bonus[i]:            # Проверяем что элемент строка str и есть подстрока руб.
        bonus[i] = bonus[i][:bonus[i].find('руб.')]                 # Оставляем тоолько сумму и убираем руб
        bonus[i] = bonus[i].replace(' ', '')                        # Удаляем лишние пробелы в строке
        bonus[i] = float(bonus[i])                                  # Переводим строку в float
        bonus[i] / 1000                                             # Делим оставшееся число на 1000
    elif isinstance(bonus[i],str) and '%' in bonus[i]:              # Проверяем что элемент строка str и есть подстрока %.
        bonus[i] = bonus[i][:bonus[i].find('%')]                    # Оставляем тоолько число и убираем %
        bonus[i] = bonus[i].replace(' ', '')                        # Удаляем лишние пробелы в строке
        bonus[i] = float(bonus[i])                                  # Переводим строку в float
        bonus[i] = bonus[i] / 100 * coast[1]                        # Делим на 100 (так как это проценты) и Умножаем на значение сделки
        bonus[i] = bonus[i] /1000                                   # Делим на 1000
df['Бонус агенту'] = bonus                                          # формированный массив присваиваем к столбцу

--- РЕДАКТИРОВАНИЕ БД ЗАМЕНА, УДАЛЕНИЕ ЭЛЕМЕНТОВ

In [6]:
strg = df['Площадь']                                                # вкладываем в переменную столбец площадь
def parse_area(strg):                                               # Создаем функцию для парсинга площади
    if '/' in strg:                                                 # Проверяем, есть ли символ "/" в строке
        strg = strg.split('/')[0]                                   # Если есть, разделяем строку по "/" и берем только первую часть (площадь)
    strg = re.sub(r'[^0-9.]', '', strg)                             # Убираем недопустимые символы из строки, оставляем только цифры и точку
    try:
        result = float(strg)                                        # Пытаемся преобразовать строку в число с плавающей запятой
    except ValueError:
        result = 0.0                                                # Если не удалось преобразовать, устанавливаем результат как 0.0
    return result                                                   # Возвращаем полученное значение площади

df['Площадь'] = df['Площадь'].apply(parse_area)                     # Применяем функцию parse_area к столбцу 'Площадь' в DataFrame

--- ПОДСЧЕТ ОБЩЕЙ ПЛОЩАДИ В БД В СТОЛБЦЕ ПЛОЩАДЬ

In [7]:

# Вычисляем общую площадь квартир в Дата сете
maxs = df['Площадь'].sum()
print(f'Общая площадь квартир в дата сете - {maxs} квм')

Общая площадь квартир в дата сете - 8354103.8 квм


--- НАСТРОЙКИ ОТОБРАЖЕНИЯ ГРАФИКОВ

In [8]:
rcParams['figure.figsize'] = 10, 5                                  # Увеличим дефолтный размер графиков
sns.set(style='whitegrid', palette='muted')                         # Включаем режим seaborn. Теперь даже методы от matplotlib будут отображаться в этом стиле
%config InlineBackend.figure_format = 'svg'                         # Формат графиков в svg

--- СТОИМОСТЬ ОДНОКОМНАТНЫХ КВАРТИР

In [ ]:
plot = df[df['Комнат' ] == '1']['Цена, руб.'].value_counts().iloc[:50].plot(kind='bar', label='Стоимость')  # Первые 50 частых стоимостей однокомнатных квартир
plt.title('Однокомнатные квартиры') # Название графика
plot.locator_params(nbins=50)       # Указываем число бинов

plt.xlabel('Стоимость') # Подпись оси x
plt.ylabel('Число повторений') # Подпись оси y
plt.legend() # Выводим легенду
plt.show()   # Показываем график

--- СТОИМОСТЬ ДВУХКОМНАТНЫХ КВАРТИР

In [ ]:
plot = df[df['Комнат' ] == '2']['Цена, руб.'].value_counts().iloc[:50].plot(kind='bar', label='Стоимость')  # Первые 25 частых стоимостей однокомнатных квартир
plt.title('Двухкомнатные квартиры') # Название графика
plot.locator_params(nbins=50)       # Указываем число бинов

plt.xlabel('Стоимость') # Подпись оси x
plt.ylabel('Число повторений') # Подпись оси y
plt.legend() # Выводим легенду
plt.show()   # Показываем график

--- ПРЕДЛОЖЕНИЯ ПО КВАРТИРАМ В БЛИЗИ ОТ МЕТРО

In [ ]:
# Посмотрим подробную информацию о всех доступных предложениях вблизи разных станций метро. Сортировка по количеству
df.groupby('Метро / ЖД станции')['Цена, руб.'].describe().sort_values(by='count', ascending=False)

--- Какая средняя стоимость квартир вблизи от различных станций метро?

In [ ]:
# Посмотрим подробную информацию о всех доступных предложениях вблизи разных станций метро. Сортировка по средней цене
df.groupby('Метро / ЖД станции')['Цена, руб.'].describe().sort_values(by='mean', ascending=False)

--- Наглядное представление о количестве предложений вблизи от станций метро.

In [ ]:
# Построим scatter зависимости количества предложений по станциям
plot = sns.catplot(x='Метро / ЖД станции', y='count', data=df.groupby('Метро / ЖД станции')['Цена, руб.'].describe().sort_values(by='count', ascending=False).iloc[:125].reset_index(), height=5, aspect=3);
plt.xticks(rotation=90) # Поворачиваем подписи метро на 90 градусов

plt.title('Количество предложений по станциям') # Подпись графика
plt.show() # Показываем график

--- Наглядное представление о средней цене квартир вблизи от станций метро.

In [ ]:
# Построим scatter зависимости стоимости от цены
plot = sns.catplot(x='Метро / ЖД станции', y='mean', data=df.groupby('Метро / ЖД станции')['Цена, руб.'].describe().sort_values(by='mean', ascending=False).iloc[:125].reset_index(), height=5, aspect=3);
plt.xticks(rotation=90) # Поворачиваем подписи метро на 90 градусов

plt.title('Средняя стоимость по станциям') # Подпись графика
plt.show() # Показываем график

--- Как зависит средняя цена квартиры от числа предложений?

In [ ]:
# Построим зависимость числа предложений от цены
sns.lmplot(data = df.groupby('Метро / ЖД станции')['Цена, руб.'].describe().sort_values(by='mean', ascending=False).reset_index(), x = 'mean', y='count')
plt.title('Зависимость числа предложений от цены')
plt.show()

--- Как влияет наличие балкона на стоимость?

In [ ]:
# Группируем по наличию балкона
# Целевая переменная - Цена, тыс. руб., находим среднее значение для каждой станции
data = df.groupby('Балкон')['Цена, руб.'].mean() # Средняя стоимость для квартир, разбитых на группы по типам балконов
data.plot() # Рисуем график

xticksList = data.index.values # Определяем список значений для оси x
plt.xticks(np.arange(xticksList.shape[0]), xticksList, rotation=45) # Переходим от численных подписей к категориальным. Поворачиваем текст на 45 градусов, чтобы подписи не заходили друг на друга
plt.xlabel('Тип балкона') # Подпись оси x
plt.ylabel('Средняя стоимость') # Подпись оси y
plt.title('Оценка средней стоимости по наличию балкона') # Подпись графика
plt.legend() # Выводим легенду
plt.show()   # Показываем график

--- Как влияет наличие санузла на стоимость?

In [ ]:
# Группируем по наличию санузла
# Целевая переменная - Цена, руб., находим среднее значение для каждой станции
data = df.groupby('Санузел')['Цена, руб.'].mean() # Средняя стоимость для квартир, разбитых на группы по типам санузлов
data.plot() # Рисуем график

xticksList = data.index.values  # Определяем список значений для оси x
plt.xticks(np.arange(xticksList.shape[0]), xticksList, rotation=45) # Переходим от численных подписей к категориальным. Поворачиваем текст на 45 градусов, чтобы подписи не заходили друг на друга
plt.xlabel('Тип санузла')       # Подпись оси x
plt.ylabel('Средняя стоимость') # Подпись оси y

plt.title('Оценка средней стоимости по наличию санузла') # Подпись графика
plt.legend() # Подпись графика
plt.show()   # Показываем график

--- Как площадь влияет на стоимость?

In [ ]:
# Построим scatter зависимости площади от цены
sns.lmplot(data = df, x = 'Площадь', y='Цена, руб.')
plt.title('Стоимость в зависимости от площади')
plt.show()

In [ ]:
# Построим scatter зависимости площади от цены для квартир до 300 кв. м
sns.lmplot(data = df[df['Площадь'] < 300], x = 'Площадь', y='Цена, руб.')
plt.title('Стоимость в зависимости от площади')
plt.show()

In [ ]:
# Группируем по площади и находим среднее значение для каждой станции
mean_prices = df.groupby('Площадь')['Цена, руб.'].mean()
# Ограничиваем данные для площадей от 0 до 750 с шагом 5, используя .loc
mean_prices = mean_prices.loc[:750:5]

# Построение графика
mean_prices.plot()
plt.xlabel('Площадь')
plt.ylabel('Средняя стоимость')
plt.title('Оценка средней стоимости по площади')
plt.show()

--- Допустим, нас интересуют предложения на станции метро Курская. Какие цены чаще всего встречаются для этой станции?

In [ ]:
plot = df[df['Метро / ЖД станции'] == 'Курская м.']['Цена, руб.'].value_counts().iloc[:25].plot(kind='bar', label='Стоимость')  # Первые 25 стоимостей квартир на станции метро Курская
plt.title('Квартиры на станции "Курская"') # Подпись графика
plot.locator_params(nbins=25)  # Число бинов
plt.xlabel('Стоимость')        # Подпись оси x
plt.ylabel('Число повторений') # Подпись оси y
plt.legend() # Выводим легенду
plt.show()   # Показываем график